In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('hiring_assignment_pa_dataset_020223.csv')

df.head()

,event_name,user_id,app_version,event_time,device_id,client
0,STREAM_RECEIVED,9c0bc3ee66ff78a1c29d1732b0b8bac1a3829cfb2378b2...,3.2302.0,2023-02-01 09:38:43.955 UTC,2692debd34281a4fb39b98d56a8c1105511941eb94a534...,web
1,STREAM_REQUEST,9c0bc3ee66ff78a1c29d1732b0b8bac1a3829cfb2378b2...,3.2302.0,2023-01-31 09:44:53.132 UTC,2692debd34281a4fb39b98d56a8c1105511941eb94a534...,web
2,STREAM_REQUEST,9c0bc3ee66ff78a1c29d1732b0b8bac1a3829cfb2378b2...,3.2302.0,2023-01-31 09:40:30.742 UTC,2692debd34281a4fb39b98d56a8c1105511941eb94a534...,web
3,STREAM_REQUEST,9c0bc3ee66ff78a1c29d1732b0b8bac1a3829cfb2378b2...,3.2302.0,2023-01-31 09:30:03.612 UTC,2692debd34281a4fb39b98d56a8c1105511941eb94a534...,web
4,STREAM_REQUEST,9c0bc3ee66ff78a1c29d1732b0b8bac1a3829cfb2378b2...,3.2302.0,2023-01-31 12:29:38.506 UTC,2692debd34281a4fb39b98d56a8c1105511941eb94a534...,web


Questions
1. On which day did we observe the most activity?
2. How would you define a session?
3. What is the highest number of events observed in a session?
4. What is the average session duration?

In [4]:
df.shape


(16099, 6)

In [5]:
df.describe()


,event_name,user_id,app_version,event_time,device_id,client
count,16099,16099,16099,16099,16099,16099
unique,12,6,2,13900,6,1
top,STREAM_REQUEST,b365a66a3ac91665d3f4c129b3d6e67a5a669ef9bf3704...,3.2302.0,2023-02-01 16:06:00.01 UTC,2de06634695ca07e29931d304785b6de68a69f454e12b4...,web
freq,5120,4529,10949,2,4529,16099


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16099 entries, 0 to 16098
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   event_name   16099 non-null  object
 1   user_id      16099 non-null  object
 2   app_version  16099 non-null  object
 3   event_time   16099 non-null  object
 4   device_id    16099 non-null  object
 5   client       16099 non-null  object
dtypes: object(6)
memory usage: 754.8+ KB


In [27]:
# Convert event_time to datetime
df['event_time'] = pd.to_datetime(df['event_time'])
df['event_time']

0       2023-02-01 09:38:43.955000+00:00
1       2023-01-31 09:44:53.132000+00:00
2       2023-01-31 09:40:30.742000+00:00
3       2023-01-31 09:30:03.612000+00:00
4       2023-01-31 12:29:38.506000+00:00
                      ...               
16094   2023-02-01 20:32:38.836000+00:00
16095   2023-02-01 14:01:26.416000+00:00
16096   2023-02-01 11:11:23.879000+00:00
16097   2023-02-01 14:01:46.109000+00:00
16098   2023-02-01 20:30:16.964000+00:00
Name: event_time, Length: 16099, dtype: datetime64[ns, UTC]

In [28]:
# Question 1: day with the most activity?
df['event_date'] = df['event_time'].dt.date

events_per_day = df['event_date'].value_counts()

most_active_day = events_per_day.idxmax()

most_active_day_count = events_per_day.max()

print(f"The day with the most activity is {most_active_day} with {most_active_day_count} events.")


The day with the most activity is 2023-02-01 with 10651 events.


A session can be like series of events by the same user with no more than 30 minutes between consecutive events. 

so for eg: (If there's a gap of more than 30 minutes between events, it starts a new session)
ps: the time can vary depending on the use-case, but I think 30mins is a fair enough time to end the session if there is no activity(or events)

In [31]:
# Question 2: defining a sample session

df = df.sort_values(by=['user_id', 'event_time'])


df['time_diff'] = df.groupby('user_id')['event_time'].diff()


session_threshold = pd.Timedelta(minutes=30)
df['new_session'] = (df['time_diff'] > session_threshold) | df['time_diff'].isna()


df['session_id'] = df.groupby('user_id')['new_session'].cumsum()


print(df.head())


            event_name                                            user_id  \
944     STREAM_REQUEST  3a9816578e77860c9952c8f73fd1a122d997e03d5a630f...   
7398   STREAM_RECEIVED  3a9816578e77860c9952c8f73fd1a122d997e03d5a630f...   
13783   STREAM_REQUEST  3a9816578e77860c9952c8f73fd1a122d997e03d5a630f...   
900    STREAM_RECEIVED  3a9816578e77860c9952c8f73fd1a122d997e03d5a630f...   
7259    STREAM_REQUEST  3a9816578e77860c9952c8f73fd1a122d997e03d5a630f...   

      app_version                       event_time  \
944      3.2302.0 2023-01-31 00:15:42.766000+00:00   
7398     3.2302.0 2023-01-31 00:15:43.072000+00:00   
13783    3.2302.0 2023-01-31 00:15:46.236000+00:00   
900      3.2302.0 2023-01-31 00:15:46.513000+00:00   
7259     3.2302.0 2023-01-31 00:15:51.592000+00:00   

                                               device_id client  event_date  \
944    57e0fd9d2ffa4663e45aa2d0f4b6a276afc2f69a6d3ddb...    web  2023-01-31   
7398   57e0fd9d2ffa4663e45aa2d0f4b6a276afc2f69a6d3ddb.

In [32]:
# Question 3:  highest number of events observed in a session?

events_per_session = df.groupby('session_id').size()


max_events_in_session = events_per_session.max()

print(f"The highest number of events observed in a session is {max_events_in_session}.")


The highest number of events observed in a session is 4082.


In [33]:
# Question 4 - average session duration?


session_duration = df.groupby('session_id')['event_time'].agg(['min', 'max'])

session_duration['duration'] = (session_duration['max'] - session_duration['min']).dt.total_seconds()


average_session_duration = session_duration['duration'].mean()


print(f"The average session duration is {average_session_duration / 60:.2f} minutes.")

The average session duration is 1212.45 minutes.
